# Introduction to APIs

## Part 1: What is an API 

API stands for "Application program interface." In layman's terms, it's an interface that allows you to communicate with another app, and send/recieve data to it. This is useful when someone else already built the tool you need, and you just need to use their data.

The difference between an API and a library is that a library will compute everything on your computer, while an API will just send a request to another service, and then give you back an answer.

Fortunately, a lot of APIs have Python interfaces, so you can use them as if they were libraries!

## Part 2: How to use a Python-ported API

Let's say that you find an API for something, and it has a Python package you can install!
This is great news, as it makes the work of interracting with the API EXTREMELY easy.

For an example, I will use the Indico API. This is an API for sentiment analysis - so you give it some text, and it can tell you some things about the text (such as the mood of the text, the political leanings of the person who said it, etc...). To do this, it uses something called Machine Learning, which might be discussed later. However, the idea is that they have a bunch of samples with data, and then based on those samples, make a guess on what the sentiment of a new piece of text is. 

### Part 2A: Indico API

The first step to using this API is to sign up for their website.
Some APIs make you pay, but thankfully indico gives you 10,000 free queries a month (which should be plenty)

Just go to their website: www.indico.io, click "Get Started," and then "Sign up as you go."

Now that you have an account, you can go to their documentation and look at all the features available to use! 
The documentation is available here: https://indico.io/docs

They have many python samples to get you started, but I will take you through a few as well. 

In [1]:
# The first thing you want to do is import the library
import indicoio

ModuleNotFoundError: No module named 'indicoio'

The next thing you want to do is add your config-id to the indicoio object.
However, the way that the code sample does it is insecure, since you don't want anyone else to be able to see your key.

There's a way we can get around this though. In the same folder as your code using indico, make a file called secrets.py
In secrets.py, you should only have one variable:

indico_key = "whatever_your_key_is"

You can get your key from the documentation, or from the top of your dashboard: https://indico.io/dashboard/. I've included a file "secrets_example.py" that you can look at, but make sure you change the name to secrets.py, and have the correct API key. 

And then in your code, you can get the key like this. 

In [2]:
from secrets import indico_key
indicoio.config.api_key = indico_key

And now you're all authenticated for using your indico account! Let's run some queries.

In [3]:
# Let's find the sentiment of a string
result = indicoio.sentiment_hq("Today was a very good day!")

In [4]:
print(result)

0.9157847166


This result is a bit confusing, so let's see what the [documentation](https://indico.io/docs#sentiment_hq) says:

>This function will return a number between 0 and 1. This number is a probability representing the likelihood that the analyzed text is positive or negative. Values greater than 0.5 indicate positive sentiment, while values less than 0.5 indicate negative sentiment.

So 0.91... indicates a result that has a very high probability of being positive. Let's try a few more, just to see how well it works.

In [5]:
indicoio.sentiment_hq("Our presentation was a disaster")

0.0328917056

In [6]:
indicoio.sentiment_hq("You have a nice computer")

0.9253799319

In [9]:
indicoio.sentiment_hq("I end work at 5pm")

0.5880327225

In [14]:
# We can also pass it a list, and it will return a list of results
indicoio.sentiment_hq(["How are you today?", "I'm doing great, thank you!"])

[0.7249644995000001, 0.9912720919]

Feel free to try a few more of these just to see. Some useful applications of this may include analyzing Yelp reviews of your restaurant, analyzing chats with a friend, or analyzing novels to plot out an emotional graph of how the novel turned out. 

I'll show some examples of other API calls you can make. A lot of them return dictionaries, which contain information. 

In [10]:
# First paragraph in article about Computer Science on wikipeida
paragraph = """Computer science is the study of the theory, experimentation, and engineering that form the basis for the design and use of computers. It is the scientific and practical approach to computation and its applications and the systematic study of the feasibility, structure, expression, and mechanization of the methodical procedures (or algorithms) that underlie the acquisition, representation, processing, storage, communication of, and access to information. An alternate, more succinct definition of computer science is the study of automating algorithmic processes that scale. A computer scientist specializes in the theory of computation and the design of computational systems.[1]

Its fields can be divided into a variety of theoretical and practical disciplines. Some fields, such as computational complexity theory (which explores the fundamental properties of computational and intractable problems), are highly abstract, while fields such as computer graphics emphasize real-world visual applications. Other fields still focus on challenges in implementing computation. For example, programming language theory considers various approaches to the description of computation, while the study of computer programming itself investigates various aspects of the use of programming language and complex systems. Human–computer interaction considers the challenges in making computers and computations useful, usable, and universally accessible to humans."""

In [12]:
# nobody has time to read all of that
result = indicoio.keywords(paragraph)
print(result)

{'computer': 0.09608448800000001, 'universally accessible': 0.1869326081, 'programming language theory': 0.0322087061, 'theory': 0.1262267688, 'computer graphics': 0.0517569035, 'basis': 0.0499561376, 'complexity theory': 0.0850902368, 'complex systems': 0.0350902368, 'computer science': 0.1289636064, 'experimentation': 0.0618151058, 'programming language': 0.0643414067, 'computer scientist': 0.1350902368, 'scientific': 0.10830214910000001, 'computer interaction': 0.1501730263, 'computational complexity theory': 0.0322087061, 'computational complexity': 0.1012881383, 'science': 0.108400676, 'study': 0.06473109860000001, 'practical approach': 0.1388250084, 'computer programming': 0.0350902368, 'fields': 0.1012478037, 'theory of computation': 0.0665995761}


This gives us a dictionary of all the keywords in the paragraph, with the probabiity that they are important to the paragraph.
We can pass it a parameter "top_n" which will limit it to only that many keywords

In [15]:
result = indicoio.keywords(paragraph, top_n=7)
print(result)

{'computer scientist': 0.1350902368, 'theory': 0.1262267688, 'science': 0.108400676, 'practical approach': 0.1388250084, 'computer science': 0.1289636064, 'universally accessible': 0.1869326081, 'computer interaction': 0.1501730263}


The result is just a regular python dictionary, so we can loop through these the same way we would do it for any dictionary. 
This would also be interesting if passed in paragraphs from a book, see what the keywords are. Maybe make a graphic based on the probabilities

In [19]:
# Here is an example of the political analysis API
# obama_farewell.txt is the contents of Obama's farewell speech to the nation
speech = open("obama_farewell.txt", encoding="utf8").read()
result = indicoio.political(speech)
print(result)

{'Libertarian': 0.08965711300000001, 'Liberal': 0.7911154628, 'Green': 0.0136871245, 'Conservative': 0.1055402532}


In [21]:
# Unsurprisingly it is 80% Liberal. You could break it down sentence by sentence, to see which parts were most liberal
# We can try the same with Donald Trump's victory speech
speech = open("donald_victory.txt", encoding="utf8").read()
result = indicoio.political(speech)
print(result)

{'Libertarian': 0.1327656806, 'Liberal': 0.3615416288, 'Green': 0.1044029593, 'Conservative': 0.4012897909}


Somewhat surprisingly, this speech looks almost as liberal as it looks conservative. I'm not a political scientist, but using these kinds of APIs is a good way to find trends in large sections of data. For example, analyze all of Obama's speeches over time and see if Indico finds any trends, and look at the data and see if they're substantial in any way. The useful thing to note is that the computer has no biases towards certain people, so it will be better at finding trends because of that. 

In [29]:
# Last one to try: Indico can analyze images
# https://indico.io/docs#image_recognition

# Let's see if it can tell which what's in this image
image_url = "https://www.sciencenewsforstudents.org/sites/default/files/2016/06/main/articles/860-header-dog-breeds.jpg"
indicoio.image_recognition(image_url, top_n=10)

{'Afghan hound, Afghan': 0.076512441,
 'Blenheim spaniel': 0.0315301642,
 'English foxhound': 0.0778390318,
 'Saint Bernard, St Bernard': 0.2607925236,
 'Walker hound, Walker foxhound': 0.0526631102,
 'Welsh springer spaniel': 0.025802087,
 'beagle': 0.0403615013,
 'dogsled, dog sled, dog sleigh': 0.15499596300000001,
 'papillon': 0.0283743348,
 'pug, pug-dog': 0.0303313788}

I don't know if these breeds are the real ones, but if they are then this is impressive! Try with your own images.

## Exercises

### Question 1

Use the personality API on Indico to find the most likely traits of Donald Trump and Barack Obama, given their speeches. Does the result surprise you?

In [1]:
def get_personality(text):
    pass

### Question 2

Do something interesting with the API! It's up to you to find some corpus of text, and find out something interesting about it. 